In [1]:
import pandas as pd
import dspy
from dspy.evaluate import SemanticF1
import json
import requests
import boto3

In [2]:
# Helper functions to setup datset and evaluation
def create_dataset(path: str) -> list[dspy.Example]:
	dataset = pd.read_excel(path)
	dataset_dict = dataset.to_dict(orient='records')
	dspy_dataset = []

	for row in dataset_dict:
		dspy_dataset.append(dspy.Example(question=row['input'], response=row['expected_output']).with_inputs("question"))

	print(f"Dataset length: {len(dspy_dataset)}")
	return dspy_dataset

def create_sets(dataset: list[dspy.Example], metric: dspy.Module = SemanticF1()):
	trainset, valset, devset, testset = dataset[:15], dataset[15:30], dataset[30:40], dataset[40:]
	evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=24, display_progress=True, display_table=3)

	for name, set in zip([trainset, valset, devset, testset], ["trainset", "valset", "devset", "testset"]):
		print(f"{set} length: {len(name)}")

	return trainset, valset, devset, testset, evaluate

In [3]:
dspy_dataset = create_dataset("synthetics/synthetic_dataset_revised.xlsx")
trainset, valset, devset, testset, evaluate = create_sets(dspy_dataset)

# Save the testset to a file
with open("testset.json", "w") as f:
	json.dump([example.toDict() for example in testset], f)

Dataset length: 52
trainset length: 15
valset length: 15
devset length: 10
testset length: 12


In [4]:
def search(query: str, top_k: int) -> list[str]:
    url = "http://greencompute-1575332443.us-east-1.elb.amazonaws.com/api/llm/retrieval"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    data = {
        "query": query,
        "top_k": top_k
    }

    documents = requests.post(url, headers=headers, json=data).json()["documents"]
    return [f"[{i}]" + doc["doc_title"] + doc["url"] + "\n\n" + doc["content"] for i, doc in enumerate(documents)]

class TitanLM(dspy.LM):
    def __init__(self, model: str, client, max_tokens: int = 1024, temperature: float = 0.3, top_p: float = 0.7, **kwargs):
        self.client = client
        self.history = []
        self.temperature = temperature
        self.max_tokens = 1024
        self.top_p = top_p

        super().__init__(model, **kwargs)
        self.model = model
    
    def _format_message(self, prompt: str):
        body = json.dumps(
            {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": self.max_tokens,
                    "stopSequences": [],
                    "temperature": self.temperature,
                    "topP": self.top_p,
                },
            }
        )
        return body

    def generate_content(self, prompt: str) -> str:
        body = self._format_message(prompt)
        response = self.client.invoke_model(
            body=body,
            modelId=self.model,
            accept="application/json",
            contentType="application/json",
        )
        response_body = json.loads(response.get("body").read())
        return response_body.get("results")

    def __call__(self, prompt=None, messages=None, **kwargs):
        # Custom chat model working for text completion model
        prompt = '\n\n'.join([x['content'] for x in messages] + ['BEGIN RESPONSE:'])

        completions = self.generate_content(prompt)
        self.history.append({"prompt": prompt, "completions": completions})

        # Must return a list of strings
        return [completions[0].get("outputText")]

    def inspect_history(self):
        for interaction in self.history:
            print(f"Prompt: {interaction['prompt']} -> Completions: {interaction['completions']}")

In [5]:
class RAG(dspy.Module):
    def __init__(self, num_docs=20):
        self.num_docs = num_docs
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question, top_k=self.num_docs)
        return self.respond(context=context, question=question)

In [6]:
lm = TitanLM("amazon.titan-text-premier-v1:0", client=boto3.client("bedrock-runtime"))
dspy.configure(lm=lm)

In [7]:
rag = RAG()
rag(question="How do I increase my data center efficiency?")

Prediction(
    reasoning='The question is asking for ways to increase data center efficiency. The document "Small Data Centers, Big Energy Savings" provides tips for increasing data center efficiency.',
    response='To increase your data center efficiency, consider the following tips:\n1. Optimize your server utilization.\n2. Use energy-efficient equipment.\n3. Implement a data center infrastructure management (DCIM) system.\n4. Utilize a high-efficiency UPS system.\n5. Implement a cooling system that is optimized for your data center.'
)

In [8]:
evaluate(RAG())

Average Metric: 1.333333333333333 / 10  (13.3): 100%|██████████| 10/10 [00:19<00:00,  1.98s/it]
2024/11/20 19:47:41 INFO dspy.evaluate.evaluate: Average Metric: 1.333333333333333 / 10 (13.3%)


,question,example_response,reasoning,pred_response,SemanticF1
0,"What factors, including multi-tenancy and operator incentives, contribute to cloud's lower environmental impact?","Cloud services generally have a lower environmental impact due to several key factors: 1. **Multi-tenancy**: This concept allows multiple customers to share resources, leading to...",The context does not mention anything about cloud's lower environmental impact.,I cannot answer this question based on the given context.,
1,"Identify how virtualization, power management, and cloud computing reduce energy waste and enhance business operations in server rooms.","Virtualization reduces energy waste in server rooms by allowing multiple virtual machines to run on a single physical server, optimizing hardware usage and decreasing the...",I cannot find sufficient information to answer the question.,"Sorry, I cannot find sufficient information to answer the question.",
2,How do virtualization and cloud computing synergistically enhance server room energy efficiency and utilization rates?,Virtualization and cloud computing work together to significantly enhance server room energy efficiency and utilization rates by improving resource allocation and reducing energy consumption. Virtualization...,I cannot find sufficient information to answer the question.,"Sorry, I cannot find sufficient information to answer the question.",


13.33

In [9]:
metric = SemanticF1()
tp = dspy.MIPROv2(metric=metric, auto="medium", num_threads=24)  # use fewer threads if your rate limit is small

optimized_rag = tp.compile(
    RAG(), 
    trainset=trainset, 
    valset=valset,
	max_bootstrapped_demos=2,
 	max_labeled_demos=2,
  	requires_permission_to_run=False
)

2024/11/20 19:48:45 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 15

2024/11/20 19:48:45 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2024/11/20 19:48:45 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2024/11/20 19:48:45 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


 87%|████████▋ | 13/15 [02:37<00:24, 12.10s/it]


Bootstrapped 2 full traces after 13 examples for up to 1 rounds, amounting to 13 attempts.
Bootstrapping set 4/19


  7%|▋         | 1/15 [00:11<02:36, 11.15s/it]


...

Exception occurred: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many tokens per min, please wait before trying again.


Average Metric: 0.8333333333333334 / 1  (83.3):   7%|▋         | 1/15 [00:14<03:21, 14.39s/it]2024/11/20 20:21:30 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 9', 'Predictor 0: Few-Shot Set 7'].
2024/11/20 20:21:30 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [13.7, 58.83, 59.08, 62.03, 55.28, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2024/11/20 20:21:30 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 62.03
2024/11/20 20:21:30 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2024/11/20 20:21:30 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====


Exception occurred: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many tokens per min, please wait before trying again.


  0%|          | 0/15 [00:00<?, ?it/s]2024/11/20 20:22:04 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 17', 'Predictor 0: Few-Shot Set 17'].
2024/11/20 20:22:04 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [13.7, 58.83, 59.08, 62.03, 55.28, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2024/11/20 20:22:04 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 62.03
2024/11/20 20:22:04 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2024/11/20 20:22:04 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 62.03!


Exception occurred: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many tokens per min, please wait before trying again.


In [10]:
optimized_rag.save("output/optimized_rag_v2.json")